In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from tqdm import tqdm
import os
from itables import show
os.chdir("../../..")

from scripts import agregation

ENTREPOT_PATH = 'C:/Users/lubaude/Desktop/utils/data/'
DATA_TEST_PATH = './tests/data/test_get_aggreged_from_utilisation_intrant_synthetise/'

In [2]:

def import_df(df_name, path_data, sep, df):
    """
        importe un dataframe au chemin path_data+df_name+'.csv' et le stock dans le dictionnaire 'df' à la clé df_name
    """
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data,  df, sep = ','):
    """
        stocke dans le dictionnaire df tous les dataframes indiqués dans la liste df_names
    """
    for df_name in df_names : 
        import_df(df_name, path_data, sep, df)
    return df

df_names =  [   
                'utilisation_intrant_synthetise', 'action_synthetise', 
                'intervention_synthetise', 'connection_synthetise', 
                'noeuds_synthetise', 'plantation_perenne_phases_synthetise',
                'plantation_perenne_synthetise','synthetise',
                'sdc','dispositif'
            ]

donnees = import_dfs(df_names, ENTREPOT_PATH, {}, sep = ',')

C:\Users\lubaude\AppData\Local\Temp\ipykernel_20976\802199528.py:5: DtypeWarning: Columns (7,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep).replace({'\r\n': '\n'}, regex=True)
C:\Users\lubaude\AppData\Local\Temp\ipykernel_20976\802199528.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep).replace({'\r\n': '\n'}, regex=True)


In [3]:
studied_ids = [
    'fr.inra.agrosyst.api.entities.action.PotInputUsage_48d0a5ed-664b-42b1-973a-9fe8ae0128e1',
    'fr.inra.agrosyst.api.entities.action.SubstrateInputUsage_203eb5a3-1595-49f4-8346-d57f8df70e60',
    'fr.inra.agrosyst.api.entities.action.MineralProductInputUsage_02c0defa-de5a-4dd4-b728-4c9f6cd60414',
    'fr.inra.agrosyst.api.entities.action.PesticideProductInputUsage_962ab621-036b-46ba-be7d-8a724dc232f8',
    'fr.inra.agrosyst.api.entities.action.PesticideProductInputUsage_1528c966-b66a-4b98-8c17-65e85d832534',
    'fr.inra.agrosyst.api.entities.action.MineralProductInputUsage_0003910f-773a-470f-8b44-ee2a208dfcf1',
    'fr.inra.agrosyst.api.entities.action.BiologicalProductInputUsage_d0c126d6-81e0-48a5-b150-588909731d23',
    'fr.inra.agrosyst.api.entities.action.SeedSpeciesInputUsage_145638dd-ae28-4b3f-bb9a-1bcdf79e5b6a',
    'fr.inra.agrosyst.api.entities.action.OtherProductInputUsage_000acbb1-51ab-4f5b-8728-cb7821afb532',
    'fr.inra.agrosyst.api.entities.action.OrganicProductInputUsage_00087c4c-7862-4bce-ab4a-3eaad7562b86'
]

In [4]:
# dict : 
# cle : nom de la table a filtrer
# value : liste 0 : table qui permet de filtrer (echelle plus basse)
#           liste 1 : colonne qui permet de filtrer de element 0
lien_entre_table = {'action_synthetise' : ['utilisation_intrant_synthetise','action_synthetise_id'],
                    'intervention_synthetise' : ['action_synthetise','intervention_synthetise_id'],
                    'connection_synthetise' : ['intervention_synthetise','connection_synthetise_id'],
                    'plantation_perenne_phases_synthetise' : ['intervention_synthetise','plantation_perenne_phases_synthetise_id'],
                    'noeuds_synthetise' : ['connection_synthetise','cible_noeuds_synthetise_id'],
                    'plantation_perenne_synthetise' : ['plantation_perenne_phases_synthetise','plantation_perenne_synthetise_id'],
                    'synthetise' : [['plantation_perenne_synthetise','noeuds_synthetise'],'synthetise_id'],
                    'sdc' : ['synthetise','sdc_id'],
                    'dispositif' : ['sdc','dispositif_id']
                    }

# dict : colonnes a garder dans le data set
# cle : nom de la table
# value : liste de colonnes a garder
colonnes_utiles = {
    'utilisation_intrant_synthetise' : ['id', 'action_synthetise_id'],
    'action_synthetise' : ['id', 'intervention_synthetise_id'],
    'intervention_synthetise' : ['id', 'connection_synthetise_id', 'plantation_perenne_phases_synthetise_id'],
    'connection_synthetise' : ['id', 'cible_noeuds_synthetise_id'],
    'plantation_perenne_phases_synthetise' : ['id', 'plantation_perenne_synthetise_id'],
    'noeuds_synthetise' : ['id', 'synthetise_id', 'culture_code'],
    'plantation_perenne_synthetise' : ['id', 'synthetise_id', 'culture_code'],
    'synthetise' : ['id', 'sdc_id'],
    'sdc' : ['id', 'campagne','dispositif_id'],
    'dispositif' : ['id', 'domaine_id']
    }


data_filtre = donnees.copy()
print(data_filtre['utilisation_intrant_synthetise'].shape)
data_filtre['utilisation_intrant_synthetise'] = data_filtre['utilisation_intrant_synthetise'].loc[data_filtre['utilisation_intrant_synthetise']['id'].isin(studied_ids)]
print(data_filtre['utilisation_intrant_synthetise'].shape)

# parcours du dict des liens entre table pour filtrer progressivement les donnees lies aux id choisis
for key in lien_entre_table.keys() :
    table_a_filtrer = key
    print(table_a_filtrer)
    table_filtre = lien_entre_table[key][0]
    colonne_pour_filtrer = lien_entre_table[key][1]

    print(data_filtre[table_a_filtrer].shape)
    
    if type(table_filtre) == list:
        temp = data_filtre[table_a_filtrer].loc[data_filtre[table_a_filtrer]['id'].isin(data_filtre[table_filtre[0]][colonne_pour_filtrer])]
        for t in table_filtre[1:len(table_filtre)] :
            temp = pd.concat([temp, data_filtre[table_a_filtrer].loc[data_filtre[table_a_filtrer]['id'].isin(data_filtre[t][colonne_pour_filtrer])]])
        data_filtre[table_a_filtrer] = temp
    else :
        data_filtre[table_a_filtrer] = data_filtre[table_a_filtrer].loc[data_filtre[table_a_filtrer]['id'].isin(data_filtre[table_filtre][colonne_pour_filtrer])]
    
    print(data_filtre[table_a_filtrer].shape)

for d in data_filtre :
    colonnes_to_keep = colonnes_utiles[d]
    data_filtre[d][colonnes_to_keep].to_csv(DATA_TEST_PATH+d+'.csv',index=False)



(1064187, 13)
(10, 13)
action_synthetise
(1273388, 29)
(10, 29)
intervention_synthetise
(1208125, 18)
(9, 18)
connection_synthetise
(95511, 6)
(4, 6)
plantation_perenne_phases_synthetise
(18408, 4)
(5, 4)
noeuds_synthetise
(76911, 8)
(4, 8)
plantation_perenne_synthetise
(18251, 20)
(5, 20)
synthetise
(20107, 8)
(9, 8)
sdc
(35002, 92)
(9, 92)
dispositif
(30202, 6)
(9, 6)


In [5]:
show(agregation.get_aggreged_from_utilisation_intrant_realise(donnees).reset_index())

show(agregation.get_aggreged_from_utilisation_intrant_realise(data_filtre).reset_index())
(agregation.get_aggreged_from_utilisation_intrant_realise(data_filtre).reset_index()).to_csv(DATA_TEST_PATH + 'res' + '.csv', index=False)

In [ ]:
res = agregation.get_aggreged_from_utilisation_intrant_realise(donnees)


,action_realise_id,intervention_realise_id,noeuds_realise_id,plantation_perenne_phases_realise_id,zone_id,culture_id,plantation_perenne_realise_id,parcelle_id,sdc_id,sdc_campagne,dispositif_id,domaine_id
id,,,,,,,,,,,,


In [14]:
res.loc[res['parcelle_id'] == 'fr.inra.agrosyst.api.entities.Plot_05721837-340b-48d9-99f8-6b76f3e9c523']
res.loc[res['sdc_id'].isna()]

,action_realise_id,intervention_realise_id,noeuds_realise_id,plantation_perenne_phases_realise_id,zone_id,culture_id,plantation_perenne_realise_id,parcelle_id,sdc_id,sdc_campagne,dispositif_id,domaine_id
id,,,,,,,,,,,,
fr.inra.agrosyst.api.entities.action.MineralProductInputUsage_00218c8c-575a-4060-98c6-2ca6dd4de573,fr.inra.agrosyst.api.entities.action.MineralFe...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.Zone_d0623d16-2a...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,NaN,fr.inra.agrosyst.api.entities.Plot_b3fbdc83-ce...,NaN,NaN,NaN,NaN
fr.inra.agrosyst.api.entities.action.MineralProductInputUsage_0041c932-8188-4dcd-8e56-a4cb542999e3,fr.inra.agrosyst.api.entities.action.AbstractA...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.Zone_b390c8c6-b3...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,NaN,fr.inra.agrosyst.api.entities.Plot_3deb3cc5-49...,NaN,NaN,NaN,NaN
fr.inra.agrosyst.api.entities.action.MineralProductInputUsage_0047bf3b-d605-4300-a179-7c39a8d7b42f,fr.inra.agrosyst.api.entities.action.AbstractA...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.Zone_22daaf3f-82...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,NaN,fr.inra.agrosyst.api.entities.Plot_9a5d3ffc-48...,NaN,NaN,NaN,NaN
fr.inra.agrosyst.api.entities.action.MineralProductInputUsage_006b8fa3-6a06-4fbe-8639-5081bb1999f0,fr.inra.agrosyst.api.entities.action.AbstractA...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.Zone_a2da8ab9-d9...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,NaN,fr.inra.agrosyst.api.entities.Plot_20ac9ae1-eb...,NaN,NaN,NaN,NaN
fr.inra.agrosyst.api.entities.action.MineralProductInputUsage_0092db77-f758-46b2-9de8-2c7bf9752c17,fr.inra.agrosyst.api.entities.action.AbstractA...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.Zone_c07d0611-c2...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,NaN,fr.inra.agrosyst.api.entities.Plot_61710dc1-d6...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
fr.inra.agrosyst.api.entities.action.PesticideProductInputUsage_bdea1b8d-7051-42ce-8264-a8fc9b295469,fr.inra.agrosyst.api.entities.action.AbstractA...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.Zone_9a667b8e-3e...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.Plot_c16a0e4a-46...,NaN,NaN,NaN,NaN
fr.inra.agrosyst.api.entities.action.PesticideProductInputUsage_f98fcd67-4bb9-4197-83b3-a4f88d6bf953,fr.inra.agrosyst.api.entities.action.AbstractA...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.Zone_bbccab8f-c0...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.Plot_111c3852-50...,NaN,NaN,NaN,NaN
fr.inra.agrosyst.api.entities.action.PesticideProductInputUsage_a7e8115f-ce61-43ee-88be-5f3d6fd45c31,fr.inra.agrosyst.api.entities.action.AbstractA...,fr.inra.agrosyst.api.entities.effective.Effect...,NaN,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.Zone_0ea16d9b-b7...,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,fr.inra.agrosyst.api.entities.effective.Effect...,fr.inra.agrosyst.api.entities.Plot_cf39617c-2f...,NaN,NaN,NaN,NaN
